## идеи дальше:
(на выбор)
1. Выбрать текст (на русском), с которым было бы интересно поработать. Попробовать вытащить все тройки предикат-субъект-объект из выбранного текста, посмотреть на результат, проинтерпретировать
2. Написать функцию,которая бы учитывала сложные сказуемые *(начинает уходить, устал думать)* как единый элемент (она похожа на функцию, которая собирает однородные дополнения, нам также нужно что-то сделать с тегами)
3. попробовать предобученные модели для других языков
4. Попробовать парсеры на предложениях с [синтаксической омонимией](https://github.com/sjut/DPO_Materials/blob/ff1341f1d82ca11a763e15d76601bd6898958323/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8F/%D1%81%D0%B8%D0%BD%D1%82_%D0%BD%D0%B5%D0%BE%D0%B4%D0%BD%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%BD%D0%BE%D1%81%D1%82%D1%8C.txt), проинтерпретировать результаты 
(для этого текст, который вы хотите анализировать, необходимо сохранить в переменную одним из удобных способов)

In [2]:
from ufal.udpipe import Model, Pipeline 

from rusenttokenize import ru_sent_tokenize

with open('../../../тексты/Vlastelin-Kolets.txt', 'r', encoding='utf-8') as f:
    ru_text = f.read()
with open('../../../тексты/Un Cavallo nella luna.txt', 'r', encoding='utf-8') as f:
    it_text = f.read()

from tqdm.notebook import tqdm

from nltk import DependencyGraph, Tree

In [3]:
UDPIPE_MODEL_FN_RUS = "model_ru.udpipe"
# !wget -O {UDPIPE_MODEL_FN_RUS} 'https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true'
UDPIPE_MODEL_FN_IT = "model_it.udpipe"
# !wget -O {UDPIPE_MODEL_FN_IT} 'https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/italian-ud-2.0-170801.udpipe?raw=true'

In [4]:
model_rus = Model.load(UDPIPE_MODEL_FN_RUS)
model_it = Model.load(UDPIPE_MODEL_FN_IT)

In [5]:
pipeline_ru = Pipeline(model_rus, 'generic_tokenizer', '','','')
pipeline_it = Pipeline(model_it, 'generic_tokenizer', '','','')

In [19]:
sents_ru = ru_sent_tokenize(ru_text)

In [20]:
sents_ru.pop()

''

In [21]:
sents_it = ru_sent_tokenize(it_text)

In [24]:
N = 400
parsed_ru = list(tqdm(map(pipeline_ru.process, sents_ru[N:N+N]), total=N))
parsed_it = list(tqdm(map(pipeline_ru.process, sents_it), total=len(sents_it)))

In [25]:
def to_depgraph(parser_result:list) -> str: 
# аргумент - это conllu-файл, который получили в результате синтаксического анализа 
    for sent in parser_result:
        # убираем коменты
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        # заменяем регистр для root
        sent = sent.replace('\troot\t', '\tROOT\t')
        yield sent

In [26]:
ru_ud = list(to_depgraph(parsed_ru))

In [27]:
print(ru_ud[100])

1	Хоббиты	ХобБИТ	NOUN	NN	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur	5	nsubj	_	_
2	из	ИЗ	ADP	IN	_	4	case	_	_
3	Западного	ЗАПАДНЫЙ	ADJ	JJL	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	4	amod	_	_
4	удела	уДЕЛО	NOUN	NN	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	1	nmod	_	_
5	говорили	ГОВОРИТЬ	VERB	VBC	Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin	0	ROOT	_	SpaceAfter=No
6	,	,	PUNCT	,	_	7	punct	_	_
7	будто	БУДТО	SCONJ	IN	_	5	obj	_	_
8	с	С	ADP	IN	_	9	case	_	_
9	вершины	верШИНА	NOUN	NN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	7	nmod	_	_
10	этой	ЭТОТ	DET	DT	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	11	det	_	_
11	башни	БАШНЯ	NOUN	NN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	9	nmod	_	_
12	видно	ВИДНО	ADV	RB	_	9	nmod	_	_
13	Море	МОРЕ	PROPN	NNP	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	12	nmod	_	SpaceAfter=No
14	;	;	PUNCT	:	_	18	punct	_	_
15	но	НО	CCONJ	CC	_	18	cc	_	SpaceAfter=No
16	,	,	PUNCT	,	_	18	punct	_	_
17	насколько	насколько	ADV	RB	_	18	advmod	_	_
18	известно	ИЗ

In [37]:
ud_graph = DependencyGraph(tree_str=ru_ud[100])
ud_tree = ud_graph.tree()

ud_tree.pretty_print()

                                                      говорили                                                          
  _______________________________________________________|__________________________                                     
 |         |                        |                                            известно                               
 |         |                        |                     __________________________|___________                         
 |         |                      будто                  |      |   |      |        |         бывал                     
 |         |                ________|______              |      |   |      |        |       ____|___________________     
 |      Хоббиты            |            вершины          |      |   |      |        |      |           |          хоббит
 |         |               |    ___________|______       |      |   |      |        |      |           |            |    
 |       удела             |

In [32]:
it_ud = list(to_depgraph(parsed_it))

In [33]:
it_ud[15]

'1\tPerchè\tPerchè\tPROPN\tNNP\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t6\tnsubj\t_\t_\n2\tti\ttI\tPRON\tPRP\tCase=Dat|Number=Sing|Person=2\t4\tnmod\t_\t_\n3\tscottano\tscottanO\tADV\tRB\t_\t4\tadvmod\t_\t_\n4\tcosì\tcosì\tPROPN\tNNP\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t1\tflat\t_\t_\n5\tle\tle\tADJ\tJJ\tAnimacy=Anim|Case=Nom|Number=Plur\t4\tamod\t_\t_\n6\tmani\tmanI\tADV\tFW\t_\t0\tROOT\t_\tSpaceAfter=No\n7\t?\t?\tPUNCT\t.\t_\t6\tpunct\t_\tSpacesAfter=\\n\n\n'

In [34]:
ud_graph = DependencyGraph(tree_str=it_ud[15])
ud_tree = ud_graph.tree()

ud_tree.pretty_print()

        mani             
  _______|______          
 |            Perchè     
 |              |         
 |             così      
 |    __________|______   
 ?   ti      scottano  le



In [35]:
list(DependencyGraph(tree_str=it_ud[15]).triples())

[(('mani', 'ADV'), 'nsubj', ('Perchè', 'PROPN')),
 (('Perchè', 'PROPN'), 'flat', ('così', 'PROPN')),
 (('così', 'PROPN'), 'nmod', ('ti', 'PRON')),
 (('così', 'PROPN'), 'advmod', ('scottano', 'ADV')),
 (('così', 'PROPN'), 'amod', ('le', 'ADJ')),
 (('mani', 'ADV'), 'punct', ('?', 'PUNCT'))]

In [36]:
for sent in ru_ud[120:121]:
    for elem in DependencyGraph(tree_str=sent).triples():
        print(elem)

(('окутано', 'VERB'), 'iobj', ('тайной', 'NOUN'))
(('тайной', 'NOUN'), 'amod', ('Великой', 'ADJ'))
(('окутано', 'VERB'), 'nsubj:pass', ('происхождение', 'NOUN'))
(('происхождение', 'NOUN'), 'nmod', ('обычая', 'NOUN'))
(('обычая', 'NOUN'), 'det', ('этого', 'DET'))
(('обычая', 'NOUN'), 'amod', ('странного', 'ADJ'))
(('происхождение', 'NOUN'), 'conj', ('искусства', 'NOUN'))
(('искусства', 'NOUN'), 'punct', (',', 'PUNCT'))
(('искусства', 'NOUN'), 'cc', ('или', 'CCONJ'))
(('искусства', 'NOUN'), 'punct', ('«', 'PUNCT'))
(('происхождение', 'NOUN'), 'punct', ('»', 'PUNCT'))
(('происхождение', 'NOUN'), 'punct', (',', 'PUNCT'))
(('окутано', 'VERB'), 'obl', ('именуют', 'VERB'))
(('именуют', 'VERB'), 'case', ('как', 'ADP'))
(('именуют', 'VERB'), 'obj', ('хоббиты', 'NOUN'))
(('хоббиты', 'NOUN'), 'det', ('его', 'DET'))
(('окутано', 'VERB'), 'punct', ('.', 'PUNCT'))


In [41]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [59]:
def comp_verb(ud):
    for sent in ud:
        try:
            for triple in DependencyGraph(tree_str=sent).triples():
                if triple[0][1] == 'VERB' and triple[2][1] == 'VERB':
                    if triple[1] == 'xcomp' and morph.parse(triple[2][0])[0].tag.POS == 'INFN':
                        yield triple
        except RecursionError:
            pass

In [60]:
list(comp_verb(ru_ud))

[(('хотели', 'VERB'), 'xcomp', ('знать', 'VERB')),
 (('привыкли', 'VERB'), 'xcomp', ('исчезать', 'VERB')),
 (('имели', 'VERB'), 'xcomp', ('прятаться', 'VERB')),
 (('могут', 'VERB'), 'xcomp', ('сравниться', 'VERB')),
 (('любили', 'VERB'), 'xcomp', ('принимать', 'VERB')),
 (('перебрались', 'VERB'), 'xcomp', ('понять', 'VERB')),
 (('думали', 'VERB'), 'xcomp', ('покидать', 'VERB')),
 (('освоили', 'VERB'), 'xcomp', ('быть', 'VERB')),
 (('стали', 'VERB'), 'xcomp', ('говорить', 'VERB')),
 (('стали', 'VERB'), 'xcomp', ('считать', 'VERB')),
 (('перестали', 'VERB'), 'xcomp', ('думать', 'VERB')),
 (('девать', 'VERB'), 'xcomp', ('выбросить', 'VERB')),
 (('может', 'VERB'), 'xcomp', ('Припугнуть', 'VERB')),
 (('может', 'VERB'), 'xcomp', ('статься', 'VERB')),
 (('можно', 'VERB'), 'xcomp', ('подумать', 'VERB')),
 (('надо', 'VERB'), 'xcomp', ('удирать', 'VERB')),
 (('пришлось', 'VERB'), 'xcomp', ('привыкать', 'VERB')),
 (('начали', 'VERB'), 'xcomp', ('строить', 'VERB')),
 (('научились', 'VERB'), 'xcomp